<a href="https://colab.research.google.com/github/Rohitsingh01698/Social_media_sentiment_analysis_Brainwavematrixsolution/blob/main/Twitter_sentiment_analysis_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Analyze this Twitter dataset to understand public sentiment towards specific topics, products, or events. Use natural language Analyze this Twitter CSV dataset using social media sentiment analysis processing (NLP) techniques to preprocess text data, extract sentiment scores, and visualize trends over time.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import warnings
warnings.filterwarnings('ignore')


In [ ]:
twitter_data=pd.read_csv(r"/content/twitter.csv")
twitter_data.head()


In [ ]:
twitter_data.info()

In [ ]:
twitter_data.describe()

In [ ]:
# 1. Handle Missing Values
twitter_data.isnull().sum()
twitter_data.dropna(subset=['tweet'], inplace=True)


In [ ]:
#  Remove Irrelevant Characters
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@\S+', '', tweet)
    tweet = re.sub(r'#\S+', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

twitter_data['tweet'] = twitter_data['tweet'].apply(clean_tweet)


In [ ]:
# Remove Duplicates
twitter_data.drop_duplicates(subset=['tweet'], inplace=True)


In [ ]:
# Lowercase Conversion
twitter_data['tweet'] = twitter_data['tweet'].str.lower()


In [ ]:
twitter_data.head()

In [ ]:
# Data Shape and Information
print("Data Shape:", twitter_data.shape)
display(twitter_data.info())

# Sentiment Distribution
sentiment_counts = twitter_data['label'].value_counts()
print("\nSentiment Distribution:\n", sentiment_counts)

plt.figure(figsize=(8, 6))
sentiment_counts.plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment Label')
plt.ylabel('Frequency')
plt.show()


# Text Length Analysis
twitter_data['tweet_length'] = twitter_data['tweet'].str.len()
tweet_length_stats =twitter_data['tweet_length'].describe()
print("\nTweet Length Statistics:\n", tweet_length_stats)

plt.figure(figsize=(10, 6))
plt.hist(twitter_data['tweet_length'], bins=30, color='lightgreen', edgecolor='black')
plt.title('Distribution of Tweet Lengths')
plt.xlabel('Tweet Length')
plt.ylabel('Frequency')
plt.show()



In [ ]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Download the 'punkt_tab' data
nltk.download('punkt_tab')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')


def preprocess_tweet(tweet):
    # Lowercasing
    tweet = tweet.lower()

    # Tokenization
    tokens = word_tokenize(tweet)

    # Stop Word Removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return " ".join(stemmed_tokens)


twitter_data['processed_tweets'] = twitter_data['tweet'].apply(preprocess_tweet)
display(twitter_data.head())

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:

def preprocess_tweet(tweet):
    # Lowercasing
    tweet = tweet.lower()


    # Tokenization
    tokens = word_tokenize(tweet)

    # Stop Word Removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return " ".join(stemmed_tokens)


twitter_data['processed_tweets'] = twitter_data['tweet'].apply(preprocess_tweet)
display(twitter_data.head())

In [ ]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import matplotlib.pyplot as plt

# 1. Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()
twitter_data['sentiment_score'] = twitter_data['processed_tweets'].apply(lambda tweet: analyzer.polarity_scores(tweet)['compound'])


In [ ]:
# 1. Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()
twitter_data['sentiment_score'] = twitter_data['processed_tweets'].apply(lambda tweet: analyzer.polarity_scores(tweet)['compound'])

# 2. Extract Features
def extract_hashtags(tweet):
    return re.findall(r'#\w+', tweet)

def extract_mentions(tweet):
    return re.findall(r'@\w+', tweet)

twitter_data['hashtags'] = twitter_data['tweet'].apply(extract_hashtags)
twitter_data['mentions'] =twitter_data['tweet'].apply(extract_mentions)

# 3. Explore Features
print(twitter_data[['sentiment_score', 'tweet_length']].describe())

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(twitter_data['sentiment_score'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Sentiment Scores')
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(twitter_data['tweet_length'], bins=20, color='lightgreen', edgecolor='black')
plt.title('Distribution of Tweet Lengths')
plt.xlabel('Tweet Length')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
plt.scatter(twitter_data['tweet_length'], twitter_data['sentiment_score'], alpha=0.5, color='purple')
plt.title('Sentiment Score vs. Tweet Length')
plt.xlabel('Tweet Length')
plt.ylabel('Sentiment Score')
plt.show()

display(twitter_data.head())

In [ ]:
# Attempt to convert 'id' to datetime
try:
    twitter_data['id'] = pd.to_datetime(twitter_data['id'])
    time_series_possible = True
    # Group by time and calculate rolling average sentiment
    twitter_data['rolling_sentiment'] = twitter_data.groupby('label')['sentiment_score'].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
except (ValueError, TypeError):
    time_series_possible = False
    print("The 'id' column cannot be converted to datetime objects. Time-series analysis is not possible.")


In [ ]:
# Group data by 'label' and calculate average sentiment
if 'label' in twitter_data.columns:
    sentiment_by_label = twitter_data.groupby('label')['sentiment_score'].mean().reset_index()
    display(sentiment_by_label)
else:
    print("No 'label' column found in the dataframe.")


In [ ]:
# 1. Line chart of rolling sentiment over time
plt.figure(figsize=(12, 6))
for label in twitter_data['label'].unique():
    subset = twitter_data[twitter_data['label'] == label]
    plt.plot(subset['id'], subset['rolling_sentiment'], label=f'Label {label}')
plt.title('Rolling Sentiment Over Time')
plt.xlabel('Time')
plt.ylabel('Rolling Sentiment')
plt.legend()
plt.savefig('sentiment_over_time.png')
plt.show()



In [ ]:
# 2. Bar chart of average sentiment by label
plt.figure(figsize=(8, 6))
plt.bar(sentiment_by_label['label'], sentiment_by_label['sentiment_score'], color=['skyblue', 'salmon'])
plt.title('Average Sentiment by Label')
plt.xlabel('Label')
plt.ylabel('Average Sentiment Score')
plt.savefig('sentiment_by_label.png')
plt.show()

In [ ]:

# Assuming 'label' column represents sentiment (0: Negative, 1: Positive)
positive_text = " ".join(twitter_data[twitter_data["label"] == 1]["tweet"])
# Filter using 'label' column, assuming 1 represents positive sentiment

# Now you can use WordCloud since it's imported
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(positive_text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud: Positive Tweets")
plt.show()

In [ ]:
# Plot sentiment distribution
plt.figure(figsize=(8,5))
# Assuming 'label' column represents sentiment (0: Negative, 1: Positive)
# Replace 'sentiment' with 'label' if it's the sentiment column
twitter_data["label"].value_counts().plot(kind="bar", color=["skyblue", "salmon", "lightgreen"])
plt.title("Sentiment Distribution")
plt.ylabel("Number of Tweets")
plt.xlabel("Sentiment")
plt.grid(axis="y")
plt.tight_layout()
plt.show()

Data Analysis Key Findings

Sentiment Imbalance: The dataset exhibits a significant imbalance in sentiment labels, with label 0 (likely negative) being substantially more frequent than label 1 (likely positive). Further investigation is needed to understand the implications of this imbalance.

Average Sentiment per Label: Label 0 has an average sentiment score of 0.089734, while Label 1 has -0.083558. The meaning of each label requires more context to determine the nature of the sentiment.

Rolling Sentiment Trends: A 7-day rolling average of sentiment scores was calculated to identify trends over time, allowing for the visualization of sentiment fluctuations.

Insights:

Investigate data source and potential biases: Analyze the 'id' column and any available metadata to understand the time frame covered, tweet sources, and potential biases. This is crucial for interpreting the sentiment analysis accurately.
Deep Dive into the meaning of the labels: Understand what the labels (0 and 1) represent in the context of the dataset, and look for any potential label inconsistencies.
The processed and cleaned dataset gives a clear overview of public sentiment.

This analysis can guide businesses, marketers, or researchers in understanding audience attitudes and making data-driven decisions.